# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770332857968                   -0.53    8.0         
  2   -2.771686346023       -2.87       -1.30    1.0    129ms
  3   -2.771714202631       -4.56       -2.63    1.0    109ms
  4   -2.771714710108       -6.29       -3.80    2.0    156ms
  5   -2.771714714745       -8.33       -4.18    2.0    121ms
  6   -2.771714715243       -9.30       -5.59    1.0   93.7ms
  7   -2.771714715250      -11.20       -6.19    2.0    115ms
  8   -2.771714715250      -13.36       -6.80    1.0   97.8ms
  9   -2.771714715250      -14.03       -7.12    2.0    132ms
 10   -2.771714715250      -15.05       -8.13    1.0    106ms


-0.00013457451891321168

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770546193308                   -0.53    9.0         
  2   -2.771778146879       -2.91       -1.32    1.0   94.5ms
  3   -2.771801794417       -4.63       -2.52    1.0    138ms
  4   -2.771802027346       -6.63       -3.29    1.0   91.0ms
  5   -2.771802073558       -7.34       -3.97    2.0    109ms
  6   -2.771802074385       -9.08       -4.61    1.0    122ms
  7   -2.771802074474      -10.05       -5.32    2.0    110ms
  8   -2.771802074476      -11.73       -6.51    1.0    109ms
  9   -2.771802074476      -13.29       -6.81    2.0    114ms
 10   -2.771802074476      -14.15       -7.21    1.0    124ms
 11   -2.771802074476   +  -14.65       -7.96    1.0    110ms
 12   -2.771802074476      -14.12       -8.88    2.0    134ms


0.01761222150973571

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457451891321168
Displaced dipole:  0.01761222150973571
Polarizability :   1.7746796028648921


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920840330e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551827613e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852773519686e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603232372e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088786471448e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.275901601033e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.535679061121e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.535679061121e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 7.828783250500e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.050236679832e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.665217274988e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.665217274988e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.664905806011789e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.